<a href="https://colab.research.google.com/github/yoshihiroo/programming-workshop/blob/master/QC4U_2022/qc4uchapter2_cirq_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QC4U Day2 Cirq porting
2022.10.15 updated.

This is an attempt to porting of the [QC4U](https://altema.is.tohoku.ac.jp/QC4U/) code written by Prof. Ohzeki of Tohoku University into Cirq for my recap and understanding. I am stealing with pride the almost all text of the explanation from the original site. (The article has been published with Prof. Ohzeki's permission.)

[The original code of Day 2](https://colab.research.google.com/gist/mohzeki222/3e775df4ee15de10cd5aa6912332abdb/qc4uchapter2.ipynb)

# Installing Cirq

Quantum computers are available from several companies.
One of them is Google, which thankfully provides a library for development.
Let's use the quantum computer together by using this library.

(Note added) After executing the command, the RESTART RUNTIME button will appear with WARNING, so click on it to restart the runtime.

In [ ]:
pip install cirq

In this case, we will use Grover's search algorithm to perform the calculation of increasing the amplitude and obtaining the desired quantum state.

In [ ]:
import cirq

This time, we use 5 qubits to extract one desired quantum state from the $2^5$ possible quantum states.
Grover's algorithm requires the presence of an "oracle" that requests which quantum state it wants.
We ask that oracle to check that we want this quantum state.
If the oracle checks that you want this quantum state, it is not a check, because if the check is something that increases the amplitude, it would be a fake.
The oracle is supposed to only invert the amplitude without changing the amplitude.

Let's first consider how to mark that desired state.
It might be a bit easier if we only had one qubit, so let's consider two or more.
We can take out one of the four states, namely|00>,|01>,|10>,|11>.
For example, we want to mark only the quantum state |11>.
There is a circuit that is just right for this, called a control Z-gate.

In [ ]:
n=2

qc = cirq.Circuit()
q = cirq.LineQubit.range(n)

First, we have two qubits.
We apply an Hadamard circuit to these to create a superposition of multiple states.

In [ ]:
qc.append(cirq.H.on_each(q))

If you want to see the progress along the way, run it in the simulator.

In [ ]:
sim = cirq.Simulator()
res = sim.simulate(qc)
print(res)

To display the results, the code should have been as follows.

In [ ]:
print(res.final_state_vector.round(5))

Each is 1/2, so squaring them together gives us 1/4, which means that all four states have equal probability.

Now let's apply a control z-gate to this.
You can implement it with cz.
You specify which is the control qubit and which is the target qubit and run it.

In [ ]:
qc.append(cirq.CZ(q[0], q[1]))

It was a PRINT to see the status of the circuits along the way.

In [ ]:
print(qc)

Now what is the amplitude at this point?

In [ ]:
sim = cirq.Simulator()
res = sim.simulate(qc)
print(res)

We are able to mark -1 on |11> as we wanted.
What this does is that when the control qubit is 0, it does nothing. And when the control qubit is 1, it causes a circuit called Z to act.
|When the control qubit is 1, it acts on the circuit called Z. When the control qubit is 1, it acts on the circuit called Z. When the control qubit is 0, it does nothing.
That something, the circuit Z, is supposed to remain the same when the bit is 0, and mark the bit as -1 when it is 1.
So, the result of the calculation is that the circuit does nothing for |10> and adds a marker for |11>.
It is amazing** that this can be done only once, without taking out each of |00>,|01>,|10>,|11> in parallel.

### Equivalent Circuits
There are many different types of quantum circuits, some of which give the same calculation results.
It may be difficult to memorize all of them, but as you get used to them, you will understand the calculation results, and you may think that some of them can be written in a different way.

In particular, the Hadamard circuit is famous for reversing the roles of Z and X.
Z was left as it is for 0 and marked for 1.
The Hadamardd circuit creates the superposition state of|0>+|1> for the case of|0> and the superposition state of|0>-|1> for the case of|1>.
So, if we invert using X, we get |1>-|0>, which means that the coefficient is multiplied by -1 compared to the original state. Notice that this is similar to the effect of Z.
However, this superposition is complicated as it is, so let's undo it.
Let us run the following circuit.

In [ ]:
qc2 = cirq.Circuit()
q = cirq.LineQubit.range(1)

Here, X is applied after the Hadamard circuit, followed by the Hadamard circuit.


In [ ]:
qc2.append([
    cirq.H(q[0]),
    cirq.X(q[0]),
    cirq.H(q[0])
])

The overall circuit is as follows

In [ ]:
print(qc2)

What would be the result of this calculation?

In [ ]:
res2 = sim.simulate(qc2)
print(res2.final_state_vector.round(5))

It's no wonder nothing happened. Since the initial state was |0>, nothing would have happened as it was.
This is similar to the action of Z.
Now let's invert the first state to |1>.

In [ ]:
qc3 = cirq.Circuit()
q = cirq.LineQubit.range(1)
qc3.append(cirq.X(q[0]))

Now let's run Hadamard, X, and Hadamard against this.

In [ ]:
qc3.append([
    cirq.H(q[0]),
    cirq.X(q[0]),
    cirq.H(q[0])
])

The overall circuit is as follows

In [ ]:
print(qc3)

Let's see what the results are in the simulator.

In [ ]:
res3 = sim.simulate(qc3)
print(res3.final_state_vector)

Like Z, it now does nothing when it is 0, and when it is 1, the coefficient reverses to -1.

In the same way, a control Z circuit can be made from a control X circuit.

Therefore, adding -1 to only |11> can be performed as follows.

In [ ]:
qc4 = cirq.Circuit()
q = cirq.LineQubit.range(2)

Now we first apply the Hadamard circuit to create a superposition condition.

In [ ]:
qc4.append(cirq.H.on_each(q))

Then let's run it with a control X-gate instead of a control Z-gate.

In [ ]:
qc4.append([
    cirq.H(q[1]),
    cirq.CX(q[0],q[1]),
    cirq.H(q[1])
])

The whole circuit is starting to look quite complicated looking.

In [ ]:
from cirq.contrib.svg import SVGCircuit
SVGCircuit(qc4)

Now let's run this circuit.

In [ ]:
res4 = sim.simulate(qc4)
print(res4.final_state_vector.round(5))

As hoped, we have successfully inverted the coefficients of only|11> to -1!
Thus, the Hadamard circuit can reverse the effects of X and Z.
Also, the fact that the circuit makes a superposition state of|0>+|1> when acted upon in the state of|0> suggests that it is an important circuit with characteristics typical of quantum circuits.

### Free circuit design

Not 11 when you want to mark it.
How can we mark other states?
Let's say you want to mark 01, for example.
In that case, let's try to mark 01 as 11, apply CZ, and then put it back to the original state as if nothing happened.

In [ ]:
qc5 = cirq.Circuit()
q = cirq.LineQubit.range(2)

First, we create the same superposition state and apply X to the second qubit, so that 10 becomes 11.
Now 00 will be flipped to 01 as well, but in this case we can't put a marker on it.
You can undo it later and you won't be offended.

In [ ]:
qc5.append(cirq.H.on_each(q))

In [ ]:
qc5.append([
    cirq.X(q[1]),
    cirq.CZ(q[0],q[1]),
    cirq.X(q[1])
])

The overall circuit will look something like this.

In [ ]:
SVGCircuit(qc5)

Now what are the results of this execution?

In [ ]:
res5 = sim.simulate(qc5)
print(res5.final_state_vector.round(5))
print(res5)

We were able to mark 10 (third) as desired. In this way we can realize the oracle by using control Z to mark the oracle.
Similar to the control Z in the case of 2 qubits, in the case of multiple qubits, it is basically the same as the control Z in the case of|11.... .11>, we notice that we can do the equivalent if we can mark the states.

### Oracle for multiple qubits

For example, we have five qubits and prepare to search for which of the enormous number of quantum states is the best in a superposition of states.

In [ ]:
n = 5
qc6 = cirq.Circuit()
q = cirq.LineQubit.range(n)

Applying an Hadamard circuit to each of these five qubits will create a superposition state of 0s and 1s.
Since it is hard to write several of them, the for statement, the program can ask the computer to repeat the process.

In [ ]:
for k in range(n):
  qc6.append(cirq.H(q[k]))

Or you can just type on_each(q) to make all qubits do the same action.

In [ ]:
#qc6.append(cirq.H.on_each(q))

In contrast, the same behavior as a control Z-gate across multiple qubits is called a multiple control Z-gate.

In [ ]:
qc6.append(cirq.Z(q[n-1]).controlled_by(*q[0:n-1]))

In [ ]:
SVGCircuit(qc6)

Now if only this circuit would invert the coefficient to -1 only for |11111>.
Let's check that.

As always, to see what the quantum state looks like, run it in the simulator.

In [ ]:
res6 = sim.simulate(qc6)
print(res6.final_state_vector.round(5))

As expected, the coefficient is applied only to |11111>.
By the way, the magnitude of the coefficients is about 0.17678 for all amplitudes.
This is consistent with the result from $2^5=32$, which is $1/\sqrt{32}\approx 0.17678$.

Putting the above together, we can create an oracle that will mark the states we want.
We want to know how many of the 32 states we want to retrieve.
Given a number, use a function that produces a binary display of that number.

In [ ]:
N = 12
binN = format(N, '05b')[::-1] # Inverted because the most significant bit is in the opposite position of Qiskit (left for Qiskit, right for Cirq)
                              # 01100 for Qiskit. 00110 for Cirq
print(binN)

By applying X to invert the qubits where one of these was 0, we can give it that wanted state of |11111>. So we can apply a multiple control Z-gate.

In [ ]:
n = 5
qc7 = cirq.Circuit()

Let's start with the superposition.
Apply an Hadamard circuit to all the qubits.

In [ ]:
qc7.append(cirq.H.on_each(q))

In contrast, apply x where it was 0 in binN.

In [ ]:
for k in range(n):
  if binN[n-k-1] == "0":
    qc7.append(cirq.X(q[k]))

Then after that, let's execute a multiple control Z-gate.

In [ ]:
qc7.append(cirq.Z(q[n-1]).controlled_by(*q[0:n-1]))

Then perform the same action again where you applied X to restore it.

In [ ]:
for k in range(n):
  if binN[n-k-1] == "0":
    qc7.append(cirq.X(q[k]))

The overall circuit is as follows

In [ ]:
SVGCircuit(qc7)

In [ ]:
res7 = sim.simulate(qc7)
print(res7.final_state_vector)

It's too long, so the results are omitted.

We only need to confirm that the 12th coefficient is negative, so let's look at it directly.

In [ ]:
print(res7.final_state_vector[12])

We have successfully created an oracle.

Let's put together the homebrew oracle circuits we have made so far and put them in a form that we can use later.
To do this, we will create our own class in python.
While setting up exception handling with if statements to accept various numbers with a maximum value of $2^n-1$, define the following class.
Define the class as follows

In [ ]:
class Oracle(cirq.Gate):
    def __init__(self, N, n):
        if N > 2**n - 1:
          N = 2**n-1
        self.N = N
        self.n = n

    def _num_qubits_(self):
        return self.n

    def _decompose_(self, qubits):
      q = qubits
      binN = format(N, '05b')[::-1] # Inverted because the most significant bit is in the opposite position of Qiskit (left for Qiskit, right for Cirq)

      #Change the desired state to|111.... .1>.
      for k in range(self.n):
        if binN[self.n-k-1] == "0":
          yield cirq.X(q[k])

      #Multiple control Z-gate
      yield cirq.Z(q[self.n-1]).controlled_by(*q[0:self.n-1])

      #|111.... .1> to the desired state.
      for k in range(self.n):
        if binN[self.n-k-1] == "0":
          yield cirq.X(q[k])

    def _circuit_diagram_info_(self, args):
        return ["Uoracle"] * self.num_qubits()

Using this summarized circuit as soon as possible, a program can be written to make it compact as follows.

In [ ]:
n = 5
N = 8

qc8 = cirq.Circuit()
q = cirq.LineQubit.range(n)
qc8.append(cirq.H.on_each(q))
oracle_gate = Oracle(N,n)
qc8.append(oracle_gate.on(*q))


qc8.append(oracle_gate,qr)
This is a unique part of the process. It specifies which qubits you want to apply your own circuit to.
If you want to apply it to all of them, just use qr!

The resulting circuit looks like this

In [ ]:
SVGCircuit(qc8)

It has been put together in a way that I don't know what's in it, but I'm not worried because I made it myself.
However, I want to make sure that it is working properly.

In [ ]:
res8 = sim.simulate(qc8)
print(res8.final_state_vector.round(5))

In [ ]:
print(res8.final_state_vector[N].round(5))

We were able to correctly set the amplitude negative only at N!

### Creating a diffuser

Next, we will consider amplifying the amplitude of a quantum state whose amplitude is negative.
From the superposition of quantum states, only the amplitude of the quantum state we are looking for is inverted.
We can say that this is a superposition state, but a collapsed one.

For simplicity, let's create a superposition state with two qubits.

In [ ]:
n = 2
qc9 = cirq.Circuit()
q = cirq.LineQubit.range(n)
qc9.append(cirq.H.on_each(q))

When we run the simulation in this situation, all states are superimposed with the same coefficients.

In [ ]:
res9 = sim.simulate(qc9)
print(res9.final_state_vector.round(5))

What if we now solve for the superposition state?
Run the Hadamard circuit again for all qubits.

In [ ]:
qc9.append(cirq.H.on_each(q))

The entire circuit is as follows

In [ ]:
print(qc9)

The simulation shows the following changes

In [ ]:
res9 = sim.simulate(qc9)
print(res9.final_state_vector.round(5))

|00> state.
On the other hand, let's break the superposition state a bit.

In [ ]:
n = 2
qc10 = cirq.Circuit()
q = cirq.LineQubit.range(n)
qc10.append(cirq.H.on_each(q))
qc10.append(cirq.CZ(q[0],q[1]))
qc10.append(cirq.H.on_each(q))

The situation is that the amplitude was reversed only for |11> because a control Z-gate was introduced in the middle.

In [ ]:
res10 = sim.simulate(qc10)
print(res10)

Then, unlike the previous case, the probability amplitudes appear not only in the|00> but also in the other states.
First of all, if we consider that the reaction occurs when the state of|00> goes through the Hadamard circuit when the superposition is perfect, we can say that the probability amplitude leaked out to|01>,|10>,|11> due to the effect of collapsing from the superposition state.
The leaked probability amplitude must have been caused by the effect of setting the amplitude negative in order to mark the marker.
What would happen if we set only |00> to negative and leave |01>,|10>,|11> as they are and return them to their original values?
It is called putting back the overflow. Moreover, the idea is to try to change the part of the superposition state.

I could run the control Z circuit to do -1 for only the |11>.
You could have done it by adapting X to each of the qubits, changing the |00> to |11>, running the control Z circuit, and then
and then undo the change.

In [ ]:
n = 2
qc11 = cirq.Circuit()
q = cirq.LineQubit.range(n)
#Make a superposition
qc11.append(cirq.H.on_each(q))
#Mark #11.
qc11.append(cirq.CZ(q[0],q[1]))
qc11.append(cirq.H.on_each(q))

#00 to 11.
qc11.append(cirq.X.on_each(q))
#Invert #11 amplitude (originally 00)
qc11.append(cirq.CZ(q[0],q[1]))
#11 to 00 and undoing reverses the 00 amplitude
qc11.append(cirq.X.on_each(q))


In [ ]:
res11 = sim.simulate(qc11)
print(res11)

Now let's try to undo this state with an Hadamard circuit.
Has it changed somewhat?

In [ ]:
#Dissolve superpositions
qc11.append(cirq.H.on_each(q))

In [ ]:
res11 = sim.simulate(qc11)
print(res11)

Oh, oh. Only the state of |11> was retrieved....
No way. No way. Couldn't we have done it?
The idea was to reduce the number of superposition states and preserve the coefficients that leaked out because they were the effect of the marking.
Let's make our own circuit to apply this with a general number of qubits.

In [ ]:
class Diffusion(cirq.Gate):
    def __init__(self, n):
        self.n = n

    def _num_qubits_(self):
        return self.n

    def _decompose_(self, qubits):
      q = qubits

      #Run Hadamard to be able to examine the collapse of the overlapping conditions.
      yield cirq.H.on_each(q)
      #Multiple control Z-gate to set the coefficients of only|0.... .0> to set the coefficient of only|0.... .0> to|1.... .1>.
      yield cirq.X.on_each(q)
      #Multiplex control Z-gate
      yield cirq.Z(q[n-1]).controlled_by(*q[0:n-1])
      #Undo|1... .1> to|0.... .0>.
      yield cirq.X.on_each(q)
      #Apply Hadamard to solve the superposition condition
      yield cirq.H.on_each(q)

    def _circuit_diagram_info_(self, args):
        return ["Udiff"] * self.num_qubits()

Let's run an oracle (Uoracle) and diffusion (Udiff) and see what results we get.
The idea is to search for the quantum state indicated by the number n=5 qubits, N=12.

In [ ]:
n = 5
N = 8

qc12 = cirq.Circuit()
q = cirq.LineQubit.range(n)
oracle_gate = Oracle(N,n)
diff_gate = Diffusion(n)

#State of superposition
qc12.append(cirq.H.on_each(q))
qc12.append(oracle_gate.on(*q))
qc12.append(diff_gate.on(*q))

In [ ]:
res12 = sim.simulate(qc12)
print(res12)

In [ ]:
print(res12.final_state_vector[N].round(5))

We can see that only the nth one has a larger amplitude.

As the algorithm is repeated, this probability gradually increases.
For example, let's create the following quantum circuit that repeats three times.

In [ ]:
n = 5
N = 8
Tall = 3

qc13 = cirq.Circuit()
q = cirq.LineQubit.range(n)
#State of superposition
qc13.append(cirq.H.on_each(q))
oracle_gate = Oracle(N,n)
diff_gate = Diffusion(n)
for k in range(Tall):
  qc13.append(oracle_gate.on(*q))
  qc13.append(diff_gate.on(*q))


Let's run the simulation.

In [ ]:
res13 = sim.simulate(qc13)
print(res13)

Let's look at the Nth probability amplitude.

In [ ]:
print(res13.final_state_vector[N].round(5))

We can see that it is overwhelmingly large.
If the probability amplitude is this large, the state indicated by N will almost always appear when the measurement is made.

In [ ]:
qc13.append(cirq.measure(q, key='m'))
SVGCircuit(qc13)

In [ ]:
res13 = sim.run(qc13, repetitions=1000)
counts = res13.histogram(key='m')

In [ ]:
counts

Indeed, we can see that the probability of the appearance of a state corresponding to N is very high.

In [ ]:
import matplotlib.pyplot as plt

def binary_labels(num_qubits):
    return [bin(x)[2:].zfill(num_qubits) for x in range(2 ** num_qubits)]

cirq.plot_state_histogram(res13, plt.subplot(), tick_label=binary_labels(n))
plt.xticks(rotation=90)
plt.show()